In [4]:
import torch
from torch import nn
import numpy as np
import pdb
# set device
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [5]:
import csv
jp_sentences = []
en_sentences = []
with open('data/kyoto_lexicon.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter=',')
    # skip the header row
    startLooking = False
    for row in reader:
        if startLooking:
            jp_sentences.append(row[0])
            en_sentences.append(row[1])
        startLooking = True
print(jp_sentences[:5])
print(en_sentences[:5])

['102世吉田日厚貫首', '1月15日：成人祭、新年祭', '1月3日：家運隆盛、商売繁盛祈願祭', '1月7日：七種粥神事', '21世紀COEプログラム']
['the 102nd head priest, Nikko TOSHIDA', '15th January: Seijin-sai (Adult Festival), the New Year Festival', '3rd January: Prayer Festival for the prosperity of family fortunes and business', '7th January: Nanakusa-gayu shinji (a divine service for a rice porridge with seven spring herbs to insure health for the new year)', 'The 21st Century Center Of Excellence Program']


# character-by-character prediction

In [14]:
class CharacterTable:
    def __init__(self, charset):
        self.charset = charset.union(set(['<sos>', '<eos>', '<null>']))
        self.charset = frozenset(self.charset)
        self.charlist = list(self.charset)
        self.vocab_size = len(self.charlist)
    def encode(self, char):
        '''convert from character to index
        can process (nested) list of characters'''
        if type(char) is type('asdf'):
            # char is a string
            return self.charlist.index(char)
        else:
            # char is a list of strings
            return [self.encode(char) for char in char]
    def decode(self, charInd):
        '''convert from index to character
        can process (nested) list of indices'''
        if type(charInd) is type(22):
            # charInd is an int
            return self.charlist[charInd]
        else:
            # charInd is a list of ints
            return [self.encode(charInd) for charInd in charInd]
jp_chartable = CharacterTable(set(''.join(jp_sentences)))
en_chartable = CharacterTable(set(''.join(en_sentences)))
print(en_chartable.encode([['a', 'b'], ['c', 'd']]))
print(jp_chartable.decode(1234))

[[47, 125], [60, 139]]
皆


In [ ]:
import random
class CharacterPredictor(nn.Module):
    def __init__(self, chartable, embedding_dimensions=64, hidden_size=100):
        super(CharacterPredictor, self).__init__()
        self.embedding_dimensions = embedding_dimensions
        self.hidden_size = hidden_size
        self.chartable = chartable
        self.vocab_size = self.chartable.vocab_size
        self.teacher_force_ratio = teacher_force_ratio
        self.embedding = nn.Embedding(self.vocab_size, embedding_dimensions)
        self.RNN = nn.LSTM(input_size=self.embedding_dimensions, hidden_size=self.hidden_size, batch_first=True)
        self.linear = nn.Sequential(
            nn.Linear(self.hidden_size, self.vocab_size),
            nn.LogSoftMax(dim=-1)
        )
    def forward(self, seq):
        '''
        predict sequence
        seq (batch, seq) tensor of character indices
        '''
        e = self.embedding(charInd) # (batch, seq, embed)
        batch_size = charInd.shape[0]
        h = torch.zeros(1, batch_size, self.hidden_size) # (batch, seq, hidden)
        c = torch.zeros(1, batch_size, self.hidden_size) # (batch, seq, hidden)
